In [1]:
%cd /home/mbudisic/Documents/PsTuts-RAG
import os
from getpass import getpass
from dotenv import load_dotenv
from pstuts_rag.configuration import Configuration
import asyncio

import nest_asyncio
nest_asyncio.apply()


/home/mbudisic/Documents/PsTuts-RAG


In [2]:
import pstuts_rag

In [3]:
import logging
logging.basicConfig(
    level=Configuration().eva_log_level,
    format='%(asctime)s - %(levelname)s - %(funcName)s - %(message)s'
)

In [4]:
%load_ext autoreload
%autoreload 2


In [5]:
load_dotenv()


True

In [6]:
from pstuts_rag.rag_for_transcripts import *

2025-05-30 23:53:26,776 - INFO - <module> - Loaded .env file


In [7]:
Configuration().print(logging.info)

2025-05-30 23:53:28,613 - INFO - print - Configuration parameters:


2025-05-30 23:53:28,614 - INFO - print -   eva_workflow_name: EVA_workflow
2025-05-30 23:53:28,615 - INFO - print -   eva_log_level: INFO
2025-05-30 23:53:28,615 - INFO - print -   transcript_glob: ./data/dev.json:./data/test.json
2025-05-30 23:53:28,616 - INFO - print -   embedding_model: mxbai-embed-large
2025-05-30 23:53:28,617 - INFO - print -   embedding_api: ModelAPI.OLLAMA
2025-05-30 23:53:28,617 - INFO - print -   llm_api: ModelAPI.OLLAMA
2025-05-30 23:53:28,618 - INFO - print -   max_research_loops: 2
2025-05-30 23:53:28,619 - INFO - print -   llm_tool_model: qwen3:4b
2025-05-30 23:53:28,620 - INFO - print -   n_context_docs: 3


In [8]:
datastore = DatastoreManager()
datastore.add_completion_callback(lambda _: logging.warning("Loading complete."))
asyncio.create_task(datastore.from_json_globs(Configuration().transcript_glob))

2025-05-30 23:53:29,748 - INFO - _send_single_request - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-05-30 23:53:29,781 - INFO - _send_single_request - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


<Task pending name='Task-1' coro=<DatastoreManager.from_json_globs() running at /home/mbudisic/Documents/PsTuts-RAG/pstuts_rag/pstuts_rag/datastore.py:105>>

In [9]:
await datastore.wait_for_loading()

2025-05-30 23:53:30,993 - INFO - _send_single_request - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-05-30 23:53:31,531 - INFO - _send_single_request - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-05-30 23:53:32,425 - INFO - _send_single_request - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-05-30 23:53:33,118 - INFO - _send_single_request - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-05-30 23:53:34,012 - INFO - _send_single_request - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-05-30 23:53:34,277 - INFO - _send_single_request - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-05-30 23:53:35,156 - INFO - _send_single_request - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-05-30 23:53:36,044 - INFO - _send_single_request - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 

True

In [10]:
chain = create_transcript_rag_chain(datastore)

In [11]:
response = chain.invoke({"question":"What is a layer?"})

2025-05-30 23:53:51,268 - INFO - _send_single_request - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-05-30 23:54:00,302 - INFO - _send_single_request - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


In [12]:
response.pretty_print()

================================== Ai Message ==================================

<think>
Okay, the user is asking, "What is a layer?" and I need to answer based only on the provided transcript excerpts.

First, I'll look through the context to find relevant information. The first document's page_content mentions that layers are the building blocks of any image in Photoshop CC. It also compares layers to separate flat pints of glass stacked one on top of another. Each layer has separate content. There's also mention of the Layers panel where you can select and work with layers, and that clicking the Eye icon toggles the visibility of a layer.

Another document talks about the layers panel being a powerful tool to isolate individual elements, allowing work on individual parts of the document. But that's more about using layers rather than defining what a layer is.

So, the key points are: layers are building blocks, they're like stacked glass with separate content, and the Layers panel 

In [13]:
response.additional_kwargs["context"]

[Document(metadata={'video_id': 19172, 'title': 'Understand layers', 'desc': 'Learn what layers are and why they are so useful.', 'length': '00:04:44.75', 'group': 'data/test.json', 'source': 'https://images-tv.adobe.com/avp/vr/b758b4c4-2a74-41f4-8e67-e2f2eab83c6a/f810fc5b-2b04-4e23-8fa4-5c532e7de6f8/e268fe4d-e5c7-415c-9f5c-d34d024b14d8_20170727011753.1280x720at2400_h264.mp4', 'speech_start_stop_times': [[0.47, 3.41], [3.81, 9.13], [9.309999, 15.01], [15.299999, 20.57], [20.88, 23.3], [23.83, 27.93], [29.38, 32.79], [32.96, 33.92], [34.43, 40.21], [41.91, 45.37], [45.88, 49.01], [49.54, 55.130001], [55.72, 58.49], [58.72, 62.14]], 'start': 0.47, 'stop': 62.14, '_id': 63, '_collection_name': '30065fde-3368-41c8-9092-79b8646d229f'}, page_content="Layers are the building blocks of any image in Photoshop CC. So, it's important to understand, what layers are and why to use them - which we'll cover in this video. If you're following along, open this layered image from the downloadable practi